In [1]:
#r "./../../../../public/src/L4-application/BoSSSpad/bin/Release/net6.0/BoSSSpad.dll"
using System;
using System.Threading;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
//using BoSSS.XNSE_Solver;
using BoSSS.Solution.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Utils;
using System.IO;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using MomentFittingVariants = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants;
Init();

In [2]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "Popcorn2D" : buildname; //Popcorn2D  VanishingSphere2D
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "convergenceAggStokesSteadyFinal" : dbname; //Collison´2
string table_name = String.Concat(buildname, "_", dbname);

In [3]:
using BoSSS.Solution.Gnuplot;

In [ ]:
//ExecutionQueues
var myBatch = ExecutionQueues[5];
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName,myBatch);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
var myDB = BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase; 
myDB

In [ ]:
myDB.Sessions.Where(s => s.SuccessfulTermination == true)

In [ ]:
var refSessions = myDB.Sessions.Where(s => s.Timesteps.First().Grid.NumberOfCells > 60000);
refSessions

In [ ]:
BoSSSshell.WorkflowMgm.hConvergence.Update();

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns( "AgglomerationThreshold", "DGdegree:Velocity*","Grid:hMin","Grid:NoOfCells" ,"L2Error_"+ "VelocityX", "L2Error_" +"Pressure");
Tab2.Print();
Tab2.SaveToFile(table_name + $"All_summary.json");

In [ ]:
var grids = myDB.Sessions.Select(s => s.Timesteps.First().Grid.NumberOfCells);
int NCellsFinestMesh =  grids.Max(); //  57600; 
Math.Sqrt(NCellsFinestMesh)

In [ ]:
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> 
                Convert.ToInt32(RowEntries["Grid:NoOfCells"]) != NCellsFinestMesh       
                 );

In [11]:
using BoSSS.Solution.Gnuplot;
using System.Linq;

// arrange line color and types
var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
var allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
PointTypes[] myPointTypes = new PointTypes[]{ PointTypes.Diamond, PointTypes.Box,  PointTypes.LowerTriangle,PointTypes.OpenLowerTriangle };


In [ ]:
string xVarName = "Grid:hMin";
string yVarName = "L2error_VelocityX";
int dgDegree = 1;

// Filter rows depending on selection
var Tab3 = Tab2.ExtractRows((iRow,RowEntries)=> 
                //Convert.ToDouble(RowEntries["AgglomerationThreshold"]) == 0 &&
                Convert.ToDouble(RowEntries["DGdegree:Velocity*"]) == dgDegree
            );

// Sort in ascending order of AgglomerationThreshold
System.Data.DataView view = Tab3.DefaultView;
view.Sort = "AgglomerationThreshold ASC"; 
Tab3 = view.ToTable();

Tab3.Print()


In [13]:
Plot2Ddata plot = new Plot2Ddata();

In [ ]:
var plot = Tab3.ToPlot(xVarName, yVarName,  "AgglomerationThreshold");
plot.LogY = true;
plot.LogX = true;
        for (int i = 0; i < plot.dataGroups.Length; i++){
            var dataGroup = plot.dataGroups[i];
            dataGroup.Format.LineColor = allColors[i % allColors.Length];
            dataGroup.Format.PointType = myPointTypes[i % myPointTypes.Length];
        }   
plot.PlotNow()

In [ ]:
plot.SaveToGIF(yVarName + "_k" + dgDegree + "_2D.png");
plot.SavePgfplotsFile(yVarName + "_k" + dgDegree + "_2D.tex");

In [ ]:
plot.Regression()

In [ ]:
PlotTable.PlotNow()

In [ ]:
int cnt = 0;
foreach(var reg in RegressionS){
    Console.WriteLine(varNames[cnt]);
    var regB = reg.ToArray();
    for(int j = 0; j < regB.Length; j++){       
        int dgDeg = ( regB[j].Key).Last() - '0';
        Console.Write("DG degree: "+ dgDeg+ " \t");
        double slope = regB[j].Value;
        Console.Write("Slope: " +slope + "\t");

        if((slope - (dgDeg + 1) ) >= -0.2)
        Console.WriteLine("(+)"); //Console.WriteLine("Expected slope archieved");
        else
        Console.WriteLine("(-)");
    }
    cnt++;
}